In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Import neo4j DB: 2/4

Code to translate v2.7.4_PIS-model.xlsx to neo4j database. 

## Setup

In [2]:
import pandas as pd
import re
import numpy as np
import os
from IPython.display import Image, display

In [3]:
from py2neo import Graph, Node, Relationship

In [4]:
import helpers

In [5]:
from importlib import reload

Connect to graph via docker-compose link. See http://localhost:7474/browser/

In [6]:
graph = Graph(host="neo4j")

In [7]:
q = graph.run("MATCH (n) RETURN n LIMIT 10")

In [8]:
q.stats()

{}

In [9]:
node_labels = helpers.node_labels

In [10]:
from pathlib import Path

base_path = Path("..")
parsed_path = base_path / "data" / "parsed"

In [ ]:
# start from scratch
graph.delete_all()

for l in node_labels:
    try:
        graph.schema.drop_uniqueness_constraint(l, "name")
    except:
        pass    

## Constraints on node names

Constraint automatically adds an index. 

In [ ]:
for label in node_labels:
    q = f'''CREATE CONSTRAINT unique_name_{label.lower()}
        ON (node:{label}) ASSERT node.name IS UNIQUE'''
    try: 
        qr = graph.run(q)
        print(label, qr.stats()['constraints_added'])
    except Exception as e:
        print(e)

## Read in

In [11]:
path =  parsed_path / "components.tsv"
df_components = pd.read_csv(path, sep="\t")

## metabolites

In [ ]:
label = 'Metabolite'
df_metabolites = df_components[df_components['NodeLabel'] == label].copy()

In [ ]:
df_metabolites.columns

In [ ]:
df_metabolites[df_metabolites['NodeName'].duplicated()]

In [ ]:
want_columns = ['AddedBy', 'Family', 'NodeName', 
                'external_links', 'NodeDescription', 'AdditionalInfo', 
                'Process', 'ModelV', 'ModelStatus',                 
               ]

In [ ]:
f = '%s-components.tsv'%label
df_metabolites[want_columns].to_csv("../data/import/" + f, sep="\t", index=None)

In [ ]:
q = helpers.metabolite_node_query(f, label, 
                     n_name="line.NodeName"
                    )

In [ ]:
print(q)

In [ ]:
qr = graph.run(q)
if not df_metabolites.shape[0] == qr.stats()['nodes_created']:
    raise Exception

In [ ]:
df_metabolites[df_metabolites['Clade'] != df_metabolites['NodeID']]

In [ ]:
label = "MetaboliteFamily"

In [ ]:
df_metabolites_has_family = df_metabolites[df_metabolites['Family'] != df_metabolites['NodeID']]

In [ ]:
want_columns = ['AddedBy', 'Family',
                'external_links', 'NodeDescription', 'AdditionalInfo', 
                'Process', 'ModelV', 'ModelStatus']

In [ ]:
df_metabolites_family = df_metabolites_has_family.fillna('').groupby('Family').agg({
                                                                  'Family':lambda x:list(x)[0],
                                                                  'AddedBy':lambda x:list(x)[0], 
                                                                  'NodeDescription':lambda x: ','.join(x), 
                                                                  'external_links':lambda x: ','.join([s for s in x if not s=='']), 
                                                                  'AdditionalInfo':lambda x: helpers.list_to_string(x), 
                                                                  'Process':lambda x:list(x)[0], 
                                                                  'ModelV':helpers.get_latest_model, 
                                                                  'ModelStatus':helpers.get_model_status})

In [ ]:
df_metabolites_family

In [ ]:
f = '%s-components.tsv'%label
df_metabolites_family.to_csv("../data/import/" + f, sep="\t", index=None)

In [ ]:
q = helpers.metabolite_node_query(f, label, 
                     n_name="line.Family"
                    )

In [ ]:
print(q)

In [ ]:
qr = graph.run(q)
if not df_metabolites_family.shape[0] == qr.stats()['nodes_created']:
    raise Exception 

In [ ]:
# Metabolite to MetaboliteFamily edges
edge_type = 'TYPE_OF'
f = '%s-edges.tsv'%edge_type
df_metabolites_has_family[want_columns + ["NodeName"]].to_csv("../data/import/" + f, sep="\t", index=None)

In [ ]:
q = helpers.make_create_type_of_edge_query(f, edge_type, 
                           source_label="Metabolite", target_label="MetaboliteFamily",
                           source_name="line.NodeName", target_name="line.Family")

In [ ]:
print(q)

In [ ]:
qr = graph.run(q)
if not df_metabolites_has_family.shape[0] == qr.stats()['relationships_created']:
    raise Exception

## Pathogens (Foreign)

In [ ]:
df_external = df_components[df_components["NodeLabel"].isin(helpers.foreign_node_labels)].copy()

In [ ]:
df_external.columns

In [ ]:
# Species --> '-'
# Family --> 'biological classification'
# Clade --> 'species'
df_external.columns = ['identifier', 'AddedBy', '-', 'NodeLabel', 'NodeType', 
                       'classification', 'species', 'NodeID', 'NodeName', 
                       'external_links', 'NodeDescription', 'AdditionalInfo', 
                       'Process', 'ModelV', 'ModelStatus', 
                       'gmm_ocd', 'GMM_Description', 'GMM_ShortName', 'GMM_Synonyms']

In [ ]:
#manualfix
df_external.loc[df_external['species']=='oomycete', 'classification'] = 'oomycete'

In [ ]:
df_external.head()

In [ ]:
want_columns = ['AddedBy',  
                'classification', 'species', 'NodeName', 
                'external_links', 'NodeDescription', 'AdditionalInfo', 
                'Process', 'ModelV', 'ModelStatus']

In [ ]:
df_external[['NodeType', 'classification', 'species', 'NodeName']]

In [ ]:
for label, subdf in df_external.groupby('NodeLabel'):
    print(label, end='\t')
    f = '%s-components.tsv'%label
    subdf[want_columns].to_csv("../data/import/" + f, sep="\t", index=None)
    
    q = helpers.foreign_node_query(f, label, 
                     n_name="line.NodeName"
                    )
    print(q)
    qr = graph.run(q)
    if not subdf.shape[0] == qr.stats()['nodes_created']:
        raise Exception

In [ ]:
external_entities = df_external.groupby('species').agg({
                                                      'classification':lambda x:list(x)[0],
                                                      'AddedBy':lambda x:list(x)[0], 
                                                      #'NodeDescription':lambda x: ', '.join(x), 
                                                      #'AdditionalInfo':lambda x: helpers.list_to_string(x), 
                                                      'Process':lambda x:list(x)[0], 
                                                      'ModelV':helpers.get_latest_model, 
                                                      #'ModelStatus':helpers.get_model_status
                                                }).reset_index()

In [ ]:
external_entities.columns

In [ ]:
label = 'ForeignEntity'
want_columns = ['species', 'classification', 'AddedBy', 'Process', 'ModelV']
f = '%s-components.tsv'%label
external_entities[want_columns].to_csv("../data/import/" + f, sep="\t", index=None)

In [ ]:
q = helpers.foreign_node_query(f, label, 
                     n_name="line.species", 
                    )
print(q)

In [ ]:
qr = graph.run(q)
if not external_entities.shape[0] == qr.stats()['nodes_created']:
    raise Exception

In [ ]:
# ExternalXXX to external_entities edges
want_columns = ['AddedBy', 'NodeName', 'species', 'ModelV']
edge_type = 'AGENT_OF'
f = '%s-edges.tsv'%edge_type
df_external[want_columns].to_csv("../data/import/" + f, sep="\t", index=None)

In [ ]:
q = helpers.make_create_type_of_edge_query(f, edge_type, 
                           source_label="", target_label="ForeignEntity",
                           source_name="line.NodeName", target_name="line.species")
print(q)

In [ ]:
qr = graph.run(q)
if not df_external.shape[0] == qr.stats()['relationships_created']:
    raise Exception

## Process

In [ ]:
df_process = df_components[df_components['NodeLabel'].isin(['Process'
                                                                ])].copy()

In [ ]:
df_process.head()

In [ ]:
want_columns = ['AddedBy', 'NodeLabel',  
        'NodeName', 'external_links', 'NodeDescription',
       'AdditionalInfo', 'Process', 'ModelV', 'ModelStatus']


In [ ]:
label = "Process"
f = "%s-components.tsv"%label
df_process[want_columns].to_csv("../data/import/" + f, sep="\t", index=None)

In [ ]:
q = helpers.process_node_query(f, label, 
                     n_name="line.NodeName", 
                    )
print(q)

In [ ]:
qr = graph.run(q)
if not df_process.shape[0] == qr.stats()['nodes_created']:
    raise Exception

## Complexes

In [ ]:
label = 'Complex'
df_complex = df_components[df_components['NodeLabel'] == label].copy()

In [ ]:
df_complex

In [ ]:
# save node types 
print(label, "\t", df_complex.shape[0])
df_complex.to_csv(f"../data/import/{label}-components.tsv", sep="\t", index=None)

In [ ]:
query = helpers.bioelement_node_query(f"{label}-components.tsv", label, n_name="line.Family")
qr = graph.run(query)
print(label, "\t", qr.stats()['nodes_created'])

## Bio-elements

these nodes have "Family", "Clade" (msa) and "FunctionalCluster"
at this point only import Families

In [12]:
df_bioelements = df_components[df_components['NodeLabel'].isin(helpers.plant_node_labels)].copy()

In [13]:
df_bioelements[df_bioelements['NodeName'].duplicated(keep=False)].sort_values('NodeName')[['identifier', 'species', 'Family', 'Clade', 'NodeID', 'NodeName']]

,identifier,species,Family,Clade,NodeID,NodeName
335,node0795,ath,CPS,CPS,AT4G02780,CPS
336,node0796,stu,CPS,CPS,SOTUB06G034690.1.1,CPS
337,node0797,stu,CPS,CPS,SOTUB08G006560.1.1,CPS.x1
338,node0800,sly,CPS,CPS,SOLYC06G084240.2.1,CPS.x1
339,node0798,stu,CPS,CPS,SOTUB08G020310.1.1,CPS.x2
340,node0801,sly,CPS,CPS,SOLYC08G005710.3.1,CPS.x2
477,node0823,stu,GA20ox,GA20ox,SOTUB06G023200.1.1,GA20ox.x1
478,node0828,sly,GA20ox,GA20ox,SOLYC06G050110.2.1,GA20ox.x1
479,node0824,stu,GA20ox,GA20ox,SOTUB09G017720.1.1,GA20ox.x2
480,node0829,sly,GA20ox,GA20ox,SOLYC09G009110.3.1,GA20ox.x2


In [14]:
pd.value_counts(df_bioelements['NodeLabel'])

PlantCoding       858
PlantAbstract      12
PlantNonCoding     12
Name: NodeLabel, dtype: int64

In [15]:
all_species_listed = list(df_bioelements['species'].unique()); all_species_listed
all_species = all_species_listed.copy()
all_species

['ath', 'stu', 'sly', 'osa']

In [16]:
def pick_the_set(x):
    for v in x:
        if type(v) == set:
            return ",".join(list(v))
    return ''

def str_lists_to_one_list(x):
    l = []
    for sub in x:
        l += sub.split(',')
    l = [s.strip() for s in l if not s=='']
    
    return ','.join(list(set(l)))

def get_species_homologues(level, meta=False, use=False):
    
    if use:
        df = df_bioelements[df_bioelements['ModelStatus'] == 'use'].copy()
    else:
        df = df_bioelements.copy()

    df['family'] = df['Family']
    
    df_level_species  = df.groupby([level, 'species']).agg({
      'NodeID':lambda x: set(x), 
      'family':lambda x: set(x).pop(), 
    })
       
    df_level_species.reset_index(1, inplace=True)

    cols = []
    for specie in all_species:
        col = specie + '_homologues'
        cols.append(col)
        df_level_species.loc[df_level_species['species'] == specie, col] =\
                 df_level_species[df_level_species['species'] == specie]['NodeID']
    
    selector = {
        y:pick_the_set for y in cols
    }
    selector['family'] = lambda x: x[0]
        
    df_level_species = df_level_species.groupby(level).agg(selector)
    
    if meta:
        df_level = df.fillna('').groupby(level).agg({
              'AddedBy':lambda x:list(x)[0], 
              'NodeLabel':lambda x:list(x)[0], 
              'NodeDescription':lambda x: ', '.join(list(set(x))), 
              'AdditionalInfo':lambda x: helpers.list_to_string(x), 
              'Process':lambda x:list(x)[0], 
              'ModelV':helpers.get_latest_model, 
              'species':lambda x: ','.join(list(set(x))),
              'ModelStatus':helpers.get_model_status,
              'external_links':str_lists_to_one_list, 
              'gmm_ocd':str_lists_to_one_list,
              'GMM_Description':str_lists_to_one_list,
              'GMM_ShortName':str_lists_to_one_list,
              'synonyms':str_lists_to_one_list,
        })    

        df = df_level.join(df_level_species[cols])
    else:
        df = df_level_species#[cols]
    
    df.reset_index(inplace=True)
    return df

In [17]:
df_families = get_species_homologues("Family", meta=True)

In [ ]:
# save node types 
family_node_labels = []
for t, subdf in df_families.groupby("NodeLabel"):
    print(t, "\t", subdf.shape[0])
    subdf.to_csv("../data/import/%s-components.tsv"%t, sep="\t", index=None)
    family_node_labels.append(t)

In [ ]:
for t in family_node_labels:
    labels = [t, 'Family', 'Plant']
    query = helpers.bioelement_node_query("%s-components.tsv"%t, labels, n_name="line.Family")
    #query = f"MATCH (n:{t}) DELETE n"
    #print(query)
    qr = graph.run(query)
    print(t, "\t", qr.stats()['nodes_created'])

## save files

In [23]:
df_bioelements[df_bioelements['Clade'] =='NADPH-oxidase']

,identifier,AddedBy,species,NodeLabel,NodeType,Family,Clade,NodeID,NodeName,external_links,NodeDescription,AdditionalInfo,Process,ModelV,ModelStatus,gmm_ocd,GMM_Description,GMM_ShortName,synonyms
773,node0491,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT5G07390,RBOHA,gmm_ocd:ocd_all_000146,NADPH oxidase,Respiratory burst oxidase.,S:ROS,v2.5,orthology,ocd_all_000146,respiratory burst oxidase homolog A,RBOHA,"ATRBOHA, RBOHA"
774,node0492,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT1G09090,RBOHB,gmm_ocd:ocd_all_000146,NADPH oxidase,Respiratory burst oxidase: NADPH-oxidase Atrbo...,S:ROS,v2.5,orthology,ocd_all_000146,respiratory burst oxidase-like protein,RBOHB,"ATRBOHB, ATRBOHB-BETA, RBOHB"
775,node0494,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT5G51060,RBOHC,gmm_ocd:ocd_all_000146,NADPH oxidase,Respiratory burst oxidase: It is required for ...,S:ROS,v2.5,orthology,ocd_all_000146,NADPH/respiratory burst oxidase protein D,RBOHC,"ATRBOHC, RBOHC, RHD2"
776,node0493,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT5G47910,RBOHD,gmm_ocd:ocd_all_000146,NADPH oxidase,Respiratory burst oxidase: NADPH/respiratory b...,S:ROS,v2.5,use,ocd_all_000146,respiratory burst oxidase homologue D,RBOHD,"ATRBOHD, RBOHD"
777,node0496,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT1G19230,RBOHE,gmm_ocd:ocd_all_000146,NADPH oxidase,Riboflavin synthase-like superfamily protein.,S:ROS,v2.5,orthology,ocd_all_000146,Riboflavin synthase-like superfamily protein,RBOHE,RBOHE
778,node0495,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT1G64060,RBOHF,gmm_ocd:ocd_all_000146,NADPH oxidase,Respiratory burst oxidase: Interacts with Atrb...,S:ROS,v2.5,orthology,ocd_all_000146,respiratory burst oxidase protein F,RBOHF,"ATRBOHF, RBOH F, RBOHAP108, RBOHF"
779,node0498,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT4G25090,RBOHG,gmm_ocd:ocd_all_000146,NADPH oxidase,Riboflavin synthase-like superfamily protein.,S:ROS,v2.5,orthology,ocd_all_000146,Riboflavin synthase-like superfamily protein,RBOHG,RBOHG
780,node0500,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT5G60010,RBOHH,gmm_ocd:ocd_all_000146,NADPH oxidase,Ferric reductase-like transmembrane component ...,S:ROS,v2.5,orthology,ocd_all_000146,ferric reductase-like transmembrane component ...,RBOHH,"RBOHH, RbohH"
781,node0497,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT4G11230,RBOHI,gmm_ocd:ocd_all_000146,NADPH oxidase,Riboflavin synthase-like superfamily protein.,S:ROS,v2.5,orthology,ocd_all_000146,Riboflavin synthase-like superfamily protein,RBOHI,RBOHI
782,node0499,KG,ath,PlantCoding,plant_coding,NADPH-oxidase,NADPH-oxidase,AT3G45810,RBOHJ,gmm_ocd:ocd_all_000146,NADPH oxidase,Ferric reductase-like transmembrane component ...,S:ROS,v2.5,orthology,ocd_all_000146,ferric reductase-like transmembrane component ...,RBOHJ,"RBOHJ, RbohJ"


In [ ]:
file_name = parsed_path / "bio_elements.tsv"
df_bioelements.to_csv(file_name, sep="\t", index=False)

In [18]:
dfs = []
for col_name, level in [('NodeName', 'node'), ('Clade', 'clade'), ('Family', 'family')]:
    print(col_name)
    df = get_species_homologues(col_name, use=True)

    df.rename(columns={col_name:'name'}, inplace=True)
    df['level'] = level
    df.set_index(['name', 'level'], inplace=True)
    dfs.append(df)

NodeName
Clade
Family


In [19]:
translate_df = pd.concat(dfs)
translate_df = translate_df.sort_index()
translate_df.head()

ath_homologues  \
name                       level                                             
&alpha;/&beta; hydroxylase clade                                 AT3G03990   
                           family                                AT3G03990   
&beta;-carotene isomerase  clade                                             
                           family                                            
4CLL                       clade   AT5G38120,AT1G20480,AT1G20510,AT1G20500   

                                  stu_homologues sly_homologues  \
name                       level                                  
&alpha;/&beta; hydroxylase clade                                  
                           family                                 
&beta;-carotene isomerase  clade                                  
                           family                                 
4CLL                       clade                                  

                                  osa_homologues                      family  
name                       level                                              
&alpha;/&beta; hydroxylase clade                  &alpha;/&beta; hydroxylase  
                           family                 &alpha;/&beta; hydroxylase  
&beta;-carotene isomerase  clade    OS11G0587000   &beta;-carotene isomerase  
                           family   OS11G0587000   &beta;-carotene isomerase  
4CLL                       clade                                        4CLL

In [21]:
translate_df.loc['NADPH-oxidase']

,ath_homologues,stu_homologues,sly_homologues,osa_homologues,family
level,,,,,
clade,AT5G47910,,,,NADPH-oxidase
family,AT5G47910,,,,NADPH-oxidase


In [24]:
level = 'Clade'
# use = True, meta=false

  #  if use:
df = df_bioelements[df_bioelements['ModelStatus'] == 'use'].copy()
#    else:
#        df = df_bioelements.copy()

df['family'] = df['Family']

df_level_species  = df.groupby([level, 'species']).agg({
  'NodeID':lambda x: set(x), 
  'family':lambda x: set(x).pop(), 
})

df_level_species.reset_index(1, inplace=True)

cols = []
for specie in all_species:
    col = specie + '_homologues'
    cols.append(col)
    df_level_species.loc[df_level_species['species'] == specie, col] =\
             df_level_species[df_level_species['species'] == specie]['NodeID']

selector = {
    y:pick_the_set for y in cols
}
selector['family'] = lambda x: x[0]

df_level_species = df_level_species.groupby(level).agg(selector)


df = df_level_species#[cols]

df.reset_index(inplace=True)


In [26]:
df[df['Clade'] =='NADPH-oxidase']

,Clade,ath_homologues,stu_homologues,sly_homologues,osa_homologues,family
146,NADPH-oxidase,AT5G47910,,,,NADPH-oxidase


In [ ]:
file_name = parsed_path / "level_translation.tsv"
translate_df.to_csv(file_name, sep="\t")

In [ ]:
dfs = []
for col_name, level in [('NodeName', 'node'), ('Clade', 'clade'), ('Family', 'family')]:
    print(col_name)
    df = get_species_homologues(col_name, use=False)

    df.rename(columns={col_name:'name'}, inplace=True)
    df['level'] = level
    df.set_index(['name', 'level'], inplace=True)
    dfs.append(df)

In [ ]:
translate_df = pd.concat(dfs)
translate_df = translate_df.sort_index()
translate_df.head()

In [ ]:
file_name = parsed_path / "level_not_use_translation.tsv"
translate_df.to_csv(file_name, sep="\t")

# END 